## Module 4 Homework

## load Dataset

In [2]:
import pandas as pd 
import numpy as np
import matplotlib as plt

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv')

In [4]:
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [10]:
df.isnull().sum()

lead_source                 0
industry                    0
number_of_courses_viewed    0
annual_income               0
employment_status           0
location                    0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64

In [23]:
categorical= df.dtypes[df.dtypes == 'object'].index.to_list()
numerical  = df.dtypes[df.dtypes != 'object'].index.to_list()
numerical.remove('converted')

['number_of_courses_viewed',
 'annual_income',
 'interaction_count',
 'lead_score',
 'lead_source',
 'industry',
 'employment_status',
 'location']

In [24]:
df[numerical]   = df[numerical].fillna(0)
df[categorical] = df[categorical].fillna('NA')

## Split the dataset

In [25]:
from sklearn.model_selection import train_test_split

In [26]:
full_train,df_test = train_test_split(df        ,test_size=0.2  , random_state=1)
df_train  ,df_val  = train_test_split(full_train,test_size= 0.25, random_state=1)